In [1]:
import requests
from requests import Response
from pydantic import BaseModel,RootModel,Field,field_validator
import pandas as pd

def get_urldata() -> Response | None:
    json_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    try:
        responseData:Response = requests.get(json_url)
        responseData.raise_for_status()
        if responseData.status_code == 200:
            print("下載成功")
            return responseData
        else:
            print("下載失敗")
            return None
    except Exception as a:
        print(a)
        print("連線失敗")
        return None

class Site(BaseModel):
    站點名稱:str = Field(alias='sna')
    行政區:str = Field(alias='sarea')
    總車輛數:float = Field(alias='tot')
    可借:float = Field(alias='sbi')
    可還:float = Field(alias='bemp')
    時間:str = Field(alias='mday')
    
    @field_validator("總車輛數","可借","可還",mode='before')
    @classmethod
    def empty_to_zero(cls, value):
        if value == '':
            return '0.0'
        else:
            return value

class Sites(RootModel):
    root:list[Site]
    def __iter__(self):
        return iter(self.root)
    def __getitem__(self, item):
        return self.root(item)
    
def main():
    rawData:Response | None = get_urldata()
    rawstr:str = rawData.text
    datas:Sites = Sites.model_validate_json(rawstr)
    download_data:list[dict] = datas.model_dump()
    youbike_df = pd.DataFrame(download_data)
    display(youbike_df)

if __name__ == '__main__':
    main()

下載成功


,站點名稱,行政區,總車輛數,可借,可還,時間
0,YouBike2.0_捷運科技大樓站,大安區,28.0,0.0,28.0,2024-04-16 19:19:18
1,YouBike2.0_復興南路二段273號前,大安區,21.0,4.0,17.0,2024-04-16 19:19:18
2,YouBike2.0_國北教大實小東側門,大安區,16.0,3.0,13.0,2024-04-16 19:17:32
3,YouBike2.0_和平公園東側,大安區,11.0,1.0,10.0,2024-04-16 19:19:18
4,YouBike2.0_辛亥復興路口西北側,大安區,16.0,2.0,14.0,2024-04-16 19:19:18
...,...,...,...,...,...,...
1403,YouBike2.0_臺大總圖書館西南側,臺大公館校區,30.0,1.0,28.0,2024-04-16 19:19:17
1404,YouBike2.0_臺大黑森林西側,臺大公館校區,20.0,0.0,20.0,2024-04-16 19:19:14
1405,YouBike2.0_臺大獸醫館南側,臺大公館校區,24.0,1.0,22.0,2024-04-16 19:13:19
1406,YouBike2.0_臺大新體育館東南側,臺大公館校區,40.0,1.0,39.0,2024-04-16 19:19:18
